***import***

In [ ]:
import os
import sys
import time
import urllib.request
import datetime
import time
import json
import pandas as pd
import re

from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib
import requests

In [ ]:
client_id = 'client_id'
client_secret = 'client_secret'

네이버 API는 한번 호출에 100개를 반환한다.  
한번의 코드에 500개 이상을 반환하도록 구현(하지만 이 코드도 한번에 1000개 까지만 가능)  
참고자료: https://yenpa.tistory.com/3

In [ ]:
def getresult(client_id,client_secret,query,display=100,start=1,sort='sim'):
    
    encText = urllib.parse.quote(query)
    
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + \
    "&display=" + str(display) + "&start=" + str(start) + "&sort=" + sort
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    # HTTP 응답 상태코드 확인
    rescode = response.getcode()
    if(rescode==200):
        # HTTP 응답 바디 읽기
        response_body = response.read()
        response_str = response_body.decode('utf-8')
        # JSON 문자열을 Python 객체로 변환
        response_json = json.loads(response_str)
    else:
        print("Error Code:" + rescode)
    # 검색 결과 아이템 리스트 반환
    return response_json['items']

In [ ]:
def remove_tag(my_str):
    ## 태그를 지우는 함수
    p = re.compile('(<([^>]+)>)')
    return p.sub('', my_str)

def sub_html_special_char(my_str):
    ## 특수문자를 나타내는 &apos;, &quot를 실제 특수문자로 변환
    p1 = re.compile('&lt;')
    p2 = re.compile('&gt;')
    p3 = re.compile('&amp;')
    p4 = re.compile('&apos;')
    p5 = re.compile('&quot;')

    result = p1.sub('\<', my_str)
    result = p2.sub('\>', result)
    result = p3.sub('\&', result)
    result = p4.sub('\'', result)
    result = p5.sub('\"', result)
    return result

### 주제: 미국 - 네이버뉴스

In [ ]:
query = '미국'
display = 100
start = 1
sort = 'sim'

result_USA = [] 
for i in range(0,10): 
    start = 1 + 100*i # (1)
    result_usa = getresult(client_id, client_secret, query, display, start, sort) # (2)
    result_USA.append(result_usa)

result_USA

In [ ]:
# 리스트 컴프리헨션을 사용하여 이중 리스트 풀기
result_USA = [item for sublist in result_USA for item in sublist]

print(result_USA)

In [ ]:
len(result_USA)

수집한 뉴스 데이터 중 p에 맞는 링크를 찾기 위한 정규 표현식 패턴 컴파일

In [ ]:
p = re.compile('https://n.news.naver.com/.+')
links = []
for item in result_USA:
    link = item['link']
    if p.search(link):
        links.append(link)

In [ ]:
len(links)

In [ ]:
article_ids = ['newsct_article', 'articeBody']
title_id = 'title_area'

In [ ]:
titles = []
descriptions = []

# links 리스트의 각 링크를 순회하면서 웹 페이지를 가져와서 제목과 설명을 추출
for i, link in enumerate(links):
    html = urllib.request.urlopen(link) # (1)
    bs_obj = BeautifulSoup(html, 'html.parser') # (2)
    
    # 제목 추출
    title = bs_obj.find_all('h2', {'id':title_id}) # (3)
    if len(title) > 0:
        titles.append(title[0].text)
    else:
        titles.append('')
    
    # 설명 추출
    description = bs_obj.find_all('div', {'id':article_ids}) # (4)
    if len(description) > 0:
        descriptions.append(description[0].text)
    else:
        descriptions.append('')
    time.sleep(1) 

#(1) link에 지정된 URL로 HTTP 요청을 보내고 응답을 받아옴

#(2) 받아온 HTML을 BeautifulSoup 객체로 파싱

#(3) h2 태그 중에서 title_id에 해당하는 속성을 가진 요소를 찾음

#(4) div 태그 중에서 article_ids에 해당하는 속성을 가진 요소를 찾음

In [ ]:
len(descriptions)

In [ ]:
result_dict_USA = {'title': titles, 'link': links, 'description': descriptions}
df_USA = pd.DataFrame.from_dict(result_dict_USA)

In [ ]:
df_USA

**주제: 미국 - 데이터프레임 중복 제거**

In [ ]:
df_USA_1 = pd.read_csv('미국(뉴스)1-745row.csv')
df_USA_2 = pd.read_csv('미국(뉴스)2-718row.csv')
df_USA_3 = pd.read_csv('미국(뉴스)3-696row.csv')
df_USA_4 = pd.read_csv('미국(뉴스)4-697row.csv')
df_USA_5 = pd.read_csv('미국(뉴스)5-648row.csv')

In [ ]:
merged_df_USA = pd.merge(df_USA_1, df_USA_2, on=['title', 'link', 'description'], how='outer')
df_USA_12 = merged_df_USA.drop_duplicates()

In [ ]:
merged_df_USA = pd.merge(df_USA_3, df_USA_4, on=['title', 'link', 'description'], how='outer')
df_USA_34 = merged_df_USA.drop_duplicates()

In [ ]:
merged_df_USA = pd.merge(df_USA12, df_USA345, on=['title', 'link', 'description'], how='outer')
df_USA = merged_df_USA.drop_duplicates()

In [ ]:
df_USA

### 주제: 여행 - 네이버뉴스

In [ ]:
query = '여행'
display = 100
start = 1000
sort = 'sim'

result_TRIP = []
for i in range(0,10):
    start = 1 + 100*i
    result_trip = getresult(client_id, client_secret, query, display, start, sort)
    result_TRIP.append(result_trip)

result_TRIP

In [ ]:
result_TRIP = [item for sublist in result_TRIP for item in sublist]

print(result_TRIP)

In [ ]:
len(result_TRIP)

In [ ]:
p = re.compile('https://n.news.naver.com/.+')
links = []
for item in result_TRIP:
    link = item['link']
    if p.search(link):
        links.append(link)

In [ ]:
len(links)

In [ ]:
article_ids = ['newsct_article', 'articeBody']
title_id = 'title_area'

In [ ]:
titles = []
descriptions = []

for i, link in enumerate(links):
    html = urllib.request.urlopen(link)
    bs_obj = BeautifulSoup(html, 'html.parser')
    title = bs_obj.find_all('h2', {'id':title_id})
    description = bs_obj.find_all('div', {'id':article_ids})
    if len(title) > 0:
        titles.append(title[0].text)
    else:
        titles.append('')
    
    if len(description) > 0:
        descriptions.append(description[0].text)
    else:
        descriptions.append('')
    time.sleep(1) 

In [ ]:
len(descriptions)

In [ ]:
result_dict_TRIP = {'title': titles, 'link': links, 'description': descriptions}
df_TRIP = pd.DataFrame.from_dict(result_dict_TRIP)

In [ ]:
df_TRIP # 

**주제: 여행 - 데이터프레임 중복 제거**

In [ ]:
df_TRIP_1 = pd.read_csv('여행(뉴스)1-704row.csv')
df_TRIP_2 = pd.read_csv('여행(뉴스)2-666row.csv')
df_TRIP_3 = pd.read_csv('여행(뉴스)3-665row.csv')
df_TRIP_4 = pd.read_csv('여행(뉴스)4-661row.csv')
df_TRIP_5 = pd.read_csv('여행(뉴스)5-603row.csv')
df_TRIP_6 = pd.read_csv('여행(뉴스)6-567row.csv')

In [ ]:
merged_df_TRIP = pd.merge(df_TRIP_1, df_TRIP_2, on=['title', 'link', 'description'], how='outer')
df_TRIP_12 = merged_df_TRIP.drop_duplicates()

In [ ]:
merged_df_TRIP = pd.merge(df_TRIP_3, df_TRIP_4, on=['title', 'link', 'description'], how='outer')
df_TRIP_34 = merged_df_TRIP.drop_duplicates()

In [ ]:
merged_df_TRIP = pd.merge(df_TRIP_12, df_TRIP_34, on=['title', 'link', 'description'], how='outer')
df_TRIP_1234 = merged_df_TRIP.drop_duplicates()

In [ ]:
merged_df_TRIP = pd.merge(df_TRIP_5, df_TRIP_6, on=['title', 'link', 'description'], how='outer')
df_TRIP_56 = merged_df_TRIP.drop_duplicates()

In [ ]:
merged_df_TRIP = pd.merge(df_TRIP_1234, df_TRIP_56, on=['title', 'link', 'description'], how='outer')
df_TRIP = merged_df_TRIP.drop_duplicates()

In [ ]:
df_TRIP

### 주제: 네이버 - 네이버뉴스

In [ ]:
query = '네이버'
display = 100
start = 1000
sort = 'sim'

result_NAVER = []
for i in range(0,10):
    start = 1 + 100*i
    result_naver = getresult(client_id, client_secret, query, display, start, sort)
    result_NAVER.append(result_naver)

result_NAVER

In [ ]:
# 리스트 컴프리헨션을 사용하여 이중 리스트 풀기
result_NAVER = [item for sublist in result_NAVER for item in sublist]

print(result_NAVER)

In [ ]:
len(result_NAVER)

In [ ]:
p = re.compile('https://n.news.naver.com/.+')
links = []
for item in result_NAVER:
    link = item['link']
    if p.search(link):
        links.append(link)

In [ ]:
len(links)

In [ ]:
article_ids = ['newsct_article', 'articeBody']
title_id = 'title_area'

In [ ]:
titles = []
descriptions = []

for i, link in enumerate(links):
    html = urllib.request.urlopen(link)
    bs_obj = BeautifulSoup(html, 'html.parser')
    title = bs_obj.find_all('h2', {'id':title_id})
    description = bs_obj.find_all('div', {'id':article_ids})
    if len(title) > 0:
        titles.append(title[0].text)
    else:
        titles.append('')
    
    if len(description) > 0:
        descriptions.append(description[0].text)
    else:
        descriptions.append('')
    time.sleep(1) 

In [ ]:
len(descriptions)

In [ ]:
result_dict_NAVER = {'title': titles, 'link': links, 'description': descriptions}
df_NAVER = pd.DataFrame.from_dict(result_dict_NAVER)

In [ ]:
df_NAVER

**주제: 네이버 - 데이터프레임 중복 제거**

In [ ]:
df_NAVER_1 = pd.read_csv('네이버(뉴스)1-547row.csv')
df_NAVER_2 = pd.read_csv('네이버(뉴스)2-546row.csv')
df_NAVER_3 = pd.read_csv('네이버(뉴스)3-547row.csv')
df_NAVER_4 = pd.read_csv('네이버(뉴스)4-585row.csv')
df_NAVER_5 = pd.read_csv('네이버(뉴스)5-605row.csv')

In [ ]:
merged_df_NAVER = pd.merge(df_NAVER_1, df_NAVER_2, on=['title', 'link', 'description'], how='outer')
df_NAVER_12 = merged_df_NAVER.drop_duplicates()

In [ ]:
merged_df_NAVER = pd.merge(df_NAVER_3, df_NAVER_4, on=['title', 'link', 'description'], how='outer')
df_NAVER_34 = merged_df_NAVER.drop_duplicates()

In [ ]:
merged_df_NAVER = pd.merge(df_NAVER_12, df_NAVER_34, on=['title', 'link', 'description'], how='outer')
df_NAVER_1234 = merged_df_NAVER.drop_duplicates()

In [ ]:
merged_df_NAVER = pd.merge(df_NAVER_1234, df_NAVER_5, on=['title', 'link', 'description'], how='outer')
df_NAVER = merged_df_NAVER.drop_duplicates()

In [ ]:
df_NAVER

### 주제: 네이버지도 - 네이버뉴스

In [ ]:
query = '네이버지도'
display = 100
start = 1000
sort = 'sim'

result_NAVER_MAP = []
for i in range(0,10):
    start = 1 + 100*i
    result_naver_MAP = getresult(client_id, client_secret, query, display, start, sort)
    result_NAVER_MAP.append(result_naver_MAP)

result_NAVER_MAP

In [ ]:
# 리스트 컴프리헨션을 사용하여 이중 리스트 풀기
result_NAVER_MAP = [item for sublist in result_NAVER_MAP for item in sublist]

print(result_NAVER_MAP)

In [ ]:
len(result_NAVER_MAP)

In [ ]:
p = re.compile('https://n.news.naver.com/.+')
links = []
for item in result_NAVER_MAP:
    link = item['link']
    if p.search(link):
        links.append(link)

In [ ]:
len(links)

In [ ]:
article_ids = ['newsct_article', 'articeBody']
title_id = 'title_area'

In [ ]:
titles = []
descriptions = []

for i, link in enumerate(links):
    html = urllib.request.urlopen(link)
    bs_obj = BeautifulSoup(html, 'html.parser')
    title = bs_obj.find_all('h2', {'id':title_id})
    description = bs_obj.find_all('div', {'id':article_ids})
    if len(title) > 0:
        titles.append(title[0].text)
    else:
        titles.append('')
    
    if len(description) > 0:
        descriptions.append(description[0].text)
    else:
        descriptions.append('')
    time.sleep(1) 

In [ ]:
len(descriptions)

In [ ]:
result_dict_NAVER_MAP = {'title': titles, 'link': links, 'description': descriptions}
df_NAVER_MAP = pd.DataFrame.from_dict(result_dict_NAVER_MAP)

In [ ]:
df_NAVER_MAP

### 주제: 네이버쇼핑 - 네이버뉴스

In [ ]:
query = '네이버쇼핑'
display = 100
start = 1000
sort = 'sim'

result_NAVER_SHOP = []
for i in range(0,10):
    start = 1 + 100*i
    result_naver_SHOP = getresult(client_id, client_secret, query, display, start, sort)
    result_NAVER_SHOP.append(result_naver_SHOP)

result_NAVER_SHOP

In [ ]:
# 리스트 컴프리헨션을 사용하여 이중 리스트 풀기
result_NAVER_SHOP = [item for sublist in result_NAVER_SHOP for item in sublist]

print(result_NAVER_SHOP)

In [ ]:
len(result_NAVER_SHOP)

In [ ]:
p = re.compile('https://n.news.naver.com/.+')
links = []
for item in result_NAVER_SHOP:
    link = item['link']
    if p.search(link):
        links.append(link)

In [ ]:
len(links)

In [ ]:
article_ids = ['newsct_article', 'articeBody']
title_id = 'title_area'

In [ ]:
titles = []
descriptions = []

for i, link in enumerate(links):
    html = urllib.request.urlopen(link)
    bs_obj = BeautifulSoup(html, 'html.parser')
    title = bs_obj.find_all('h2', {'id':title_id})
    description = bs_obj.find_all('div', {'id':article_ids})
    if len(title) > 0:
        titles.append(title[0].text)
    else:
        titles.append('')
    
    if len(description) > 0:
        descriptions.append(description[0].text)
    else:
        descriptions.append('')
    time.sleep(1) 

In [ ]:
len(descriptions)

In [ ]:
result_dict_NAVER_SHOP = {'title': titles, 'link': links, 'description': descriptions}
df_NAVER_SHOP = pd.DataFrame.from_dict(result_dict_NAVER_SHOP)

In [ ]:
df_NAVER_SHOP

### 주제: 네이버블로그 - 네이버뉴스

In [ ]:
query = '네이버블로그'
display = 100
start = 1000
sort = 'sim'

result_NAVER_BLOG = []
for i in range(0,10):
    start = 1 + 100*i
    result_naver_BLOG = getresult(client_id, client_secret, query, display, start, sort)
    result_NAVER_BLOG.append(result_naver_BLOG)

result_NAVER_BLOG

In [ ]:
# 리스트 컴프리헨션을 사용하여 이중 리스트 풀기
result_NAVER_BLOG = [item for sublist in result_NAVER_BLOG for item in sublist]

print(result_NAVER_BLOG)

In [ ]:
len(result_NAVER_BLOG)

In [ ]:
p = re.compile('https://n.news.naver.com/.+')
links = []
for item in result_NAVER_BLOG:
    link = item['link']
    if p.search(link):
        links.append(link)

In [ ]:
len(links)

In [ ]:
article_ids = ['newsct_article', 'articeBody']
title_id = 'title_area'

In [ ]:
titles = []
descriptions = []

for i, link in enumerate(links):
    html = urllib.request.urlopen(link)
    bs_obj = BeautifulSoup(html, 'html.parser')
    title = bs_obj.find_all('h2', {'id':title_id})
    description = bs_obj.find_all('div', {'id':article_ids})
    if len(title) > 0:
        titles.append(title[0].text)
    else:
        titles.append('')
    
    if len(description) > 0:
        descriptions.append(description[0].text)
    else:
        descriptions.append('')
    time.sleep(1) 

In [ ]:
len(descriptions)

In [ ]:
result_dict_NAVER_BLOG = {'title': titles, 'link': links, 'description': descriptions}
df_NAVER_BLOG = pd.DataFrame.from_dict(result_dict_NAVER_BLOG)

In [ ]:
df_NAVER_BLOG

### 주제: 네이버클라우드 - 네이버뉴스

In [ ]:
query = '네이버클라우드'
display = 100
start = 1000
sort = 'sim'

result_NAVER_CLOUD = []
for i in range(0,10):
    start = 1 + 100*i
    result_naver_CLOUD = getresult(client_id, client_secret, query, display, start, sort)
    result_NAVER_CLOUD.append(result_naver_CLOUD)

result_NAVER_CLOUD

In [ ]:
# 리스트 컴프리헨션을 사용하여 이중 리스트 풀기
result_NAVER_CLOUD = [item for sublist in result_NAVER_CLOUD for item in sublist]

print(result_NAVER_CLOUD)

In [ ]:
len(result_NAVER_CLOUD)

In [ ]:
p = re.compile('https://n.news.naver.com/.+')
links = []
for item in result_NAVER_CLOUD:
    link = item['link']
    if p.search(link):
        links.append(link)

In [ ]:
len(links)

In [ ]:
article_ids = ['newsct_article', 'articeBody']
title_id = 'title_area'

In [ ]:
titles = []
descriptions = []

for i, link in enumerate(links):
    html = urllib.request.urlopen(link)
    bs_obj = BeautifulSoup(html, 'html.parser')
    title = bs_obj.find_all('h2', {'id':title_id})
    description = bs_obj.find_all('div', {'id':article_ids})
    if len(title) > 0:
        titles.append(title[0].text)
    else:
        titles.append('')
    
    if len(description) > 0:
        descriptions.append(description[0].text)
    else:
        descriptions.append('')
    time.sleep(1) 

In [ ]:
len(descriptions)

In [ ]:
result_dict_NAVER_CLOUD = {'title': titles, 'link': links, 'description': descriptions}
df_NAVER_CLOUD = pd.DataFrame.from_dict(result_dict_NAVER_CLOUD)

In [ ]:
df_NAVER_CLOUD